# Patient Profile & Diagnosis Trends

1. Load data

In [13]:
import pandas as pd 
import numpy as np 

df = pd.read_csv('patient_visits.csv')

df['visit_date'] = pd.to_datetime(df['visit_date'])
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'])

print(df.head(5))

print(df.info())

  visit_id patient_id visit_date date_of_birth  patient_age patient_sex  \
0  V000001    PT00001 2022-03-15    1956-07-22           66        Male   
1  V000002    PT00002 2022-04-22    1982-11-05           39      Female   
2  V000003    PT00003 2022-05-10    1978-02-14           44        Male   
3  V000004    PT00004 2023-01-18    2004-09-02           57      Female   
4  V000005    PT00005 2023-07-30    1948-03-19           75        Male   

  icd_code  cpt_code  
0      I10     12345  
1    E11.9     23456  
2  J45.909     34567  
3    N39.0     45678  
4  I25.110     56789  
<class 'pandas.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   visit_id       200 non-null    str           
 1   patient_id     200 non-null    str           
 2   visit_date     200 non-null    datetime64[us]
 3   date_of_birth  200 non-null    datetime64[us]
 4   patient

2. Patient population overview

- 2.1 Validate age

In [17]:
df['age_from_DOB'] = ((df['visit_date'] - df['date_of_birth']).dt.days / 365.25).astype(int)
diff =df['patient_age']- df['age_from_DOB']

df['age_clean'] = np.where((df['age_from_DOB']).isna() | (diff.abs() > 2),
                           df['age_from_DOB'],
                           df['patient_age']
)

df.head(10)

,visit_id,patient_id,visit_date,date_of_birth,patient_age,patient_sex,icd_code,cpt_code,age_from_DOB,age_clean
0,V000001,PT00001,2022-03-15,1956-07-22,66,Male,I10,12345,65,66
1,V000002,PT00002,2022-04-22,1982-11-05,39,Female,E11.9,23456,39,39
2,V000003,PT00003,2022-05-10,1978-02-14,44,Male,J45.909,34567,44,44
3,V000004,PT00004,2023-01-18,2004-09-02,57,Female,N39.0,45678,18,18
4,V000005,PT00005,2023-07-30,1948-03-19,75,Male,I25.110,56789,75,75
5,V000006,PT00006,2023-11-12,1988-06-21,35,Female,F32.9,67890,35,35
6,V000007,PT00007,2024-02-08,1992-08-15,31,Female,M54.5,78901,31,31
7,V000008,PT00008,2024-03-23,1970-12-01,53,Male,K21.9,89012,53,53
8,V000009,PT00009,2022-09-14,1960-04-28,62,Female,E78.5,90123,62,62
9,V000010,PT00010,2022-12-07,1985-05-30,37,Male,F41.1,91234,37,37


- 2.2 Create age bands

In [23]:
bins = [0, 17, 39, 64, 200]
labels = ['0-17', '18-39', '40-64', '65+']

df['age_band'] = pd.cut(df['age_clean'], bins=bins, labels=labels, include_lowest=True)

df.head(10)

,visit_id,patient_id,visit_date,date_of_birth,patient_age,patient_sex,icd_code,cpt_code,age_from_DOB,age_clean,age_band
0,V000001,PT00001,2022-03-15,1956-07-22,66,Male,I10,12345,65,66,65+
1,V000002,PT00002,2022-04-22,1982-11-05,39,Female,E11.9,23456,39,39,18-39
2,V000003,PT00003,2022-05-10,1978-02-14,44,Male,J45.909,34567,44,44,40-64
3,V000004,PT00004,2023-01-18,2004-09-02,57,Female,N39.0,45678,18,18,18-39
4,V000005,PT00005,2023-07-30,1948-03-19,75,Male,I25.110,56789,75,75,65+
5,V000006,PT00006,2023-11-12,1988-06-21,35,Female,F32.9,67890,35,35,18-39
6,V000007,PT00007,2024-02-08,1992-08-15,31,Female,M54.5,78901,31,31,18-39
7,V000008,PT00008,2024-03-23,1970-12-01,53,Male,K21.9,89012,53,53,40-64
8,V000009,PT00009,2022-09-14,1960-04-28,62,Female,E78.5,90123,62,62,40-64
9,V000010,PT00010,2022-12-07,1985-05-30,37,Male,F41.1,91234,37,37,18-39


- 2.3 Patient population by age

In [22]:
age_band_counts = df['age_band'].value_counts().reindex(labels)
age_band_counts

age_band
0-17       1
18-39     68
40-64    101
65+       30
Name: count, dtype: int64

3. Diagnosis analysis

- 3.1 Overall top diagnosis

In [24]:
top_icd = df['icd_code'].value_counts().head(10)
top_icd

icd_code
I10        19
J45.909    11
N39.0      11
E11.9      10
M54.5      10
E78.5      10
F32.9       9
K21.9       9
I25.10      9
G43.909     8
Name: count, dtype: int64

In [38]:

top_5 = top_icd.head(5).index
top_icd_by_age_sex = (df[df['icd_code'].isin(top_5)].groupby(['icd_code', 'age_band', 'patient_sex'])
                    .size().reset_index(name='diagnosis_events').sort_values(by='diagnosis_events', ascending=False))

top_icd_by_age_sex

,icd_code,age_band,patient_sex,diagnosis_events
4,I10,40-64,Female,6
1,E11.9,40-64,Female,6
6,I10,65+,Female,5
7,I10,65+,Male,5
10,J45.909,40-64,Male,5
17,N39.0,18-39,Female,4
9,J45.909,40-64,Female,4
19,N39.0,40-64,Female,3
13,M54.5,18-39,Female,3
15,M54.5,40-64,Female,3


4. Visit Utilization

- 4.1 Total diagnosis event per patient

In [44]:
diagnosis_events_per_patient = (df.groupby('patient_id')['icd_code'].count()
                                .reset_index(name='total_diagnosis_events').sort_values
                                (by='total_diagnosis_events', ascending=False))

diagnosis_events_per_patient

,patient_id,total_diagnosis_events
0,PT00001,9
10,PT00011,9
1,PT00002,9
17,PT00018,9
16,PT00017,9
14,PT00015,9
13,PT00014,9
12,PT00013,9
11,PT00012,9
15,PT00016,9


- 4.2 Average number of visits per patient

In [47]:
average_visits_per_patient = df.groupby('patient_id')['visit_id'].nunique().mean()
average_visits_per_patient

np.float64(1.4761904761904763)

- 4.3 High utilizers (patients with 4+ visits)

In [51]:
high_utilizers = diagnosis_events_per_patient[diagnosis_events_per_patient['total_diagnosis_events'] >= 4]
high_utilizers

,patient_id,total_diagnosis_events
0,PT00001,9
10,PT00011,9
1,PT00002,9
17,PT00018,9
16,PT00017,9
14,PT00015,9
13,PT00014,9
12,PT00013,9
11,PT00012,9
15,PT00016,9


5. Procedure (CPT) Insights

- 5.1 Top performed procedures

In [52]:
top_cpt = df['cpt_code'].value_counts().head(10)
top_cpt

cpt_code
99213    7
99214    6
99204    6
12345    4
23456    4
34567    4
45678    4
56789    4
67890    4
78901    4
Name: count, dtype: int64